Fixed-size segmentation of Audio Samples

In [ ]:
%pip install pydub

: 

In [ ]:
# Read WAV and MP3 files to array

from pydub import AudioSegment
import numpy as np
from scipy.io import wavfile
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
import plotly

file = "E:\\audio_recorder_downloads_wavs\\Tarcutta_April_May_2021\\DryA\\20210427_AAO\\20210427T060000+1000_REC.wav"

: 

below does not work - .wav files are unsupported

In [ ]:
fs, signal = wavfile.read(file)
signal = signal / (2**15)
signal_len = len(signal)
segment_size_t = 60 # segment size in seconds
segment_size = segment_size_t * fs  # segment size in samples
# Break signal into list of segments in a single-line Python code
segments = np.array([signal[x:x + segment_size] for x in
                     np.arange(0, signal_len, segment_size)])
# Save each segment in a seperate filename
for iS, s in enumerate(segments):
    wavfile.write("D:/PhD-data/tarcutta-half-day/60 sec chunks/20210427T060000+1000_REC_audio_segment_{0:d}_{1:d}.wav".format(segment_size_t * iS,
                                                              segment_size_t * (iS + 1)), fs, (s))

: 

In [ ]:
# import IPython
# # Remove pauses using an energy threshold = 50% of the median energy:
# energies = [(s**2).sum() / len(s) for s in segments]
# # (attention: integer overflow would occure without normalization here!)
# thres = 0.5 * np.median(energies)
# index_of_segments_to_keep = (np.where(energies > thres)[0])
# # get segments that have energies higher than a the threshold:
# segments2 = segments[index_of_segments_to_keep]
# # concatenate segments to signal:
# new_signal = np.concatenate(segments2)
# # and write to file:
# wavfile.write("D:/PhD-data/Wambiana/60 sec chunks/processed.wav", fs, new_signal)
# # plotly.offline.iplot({ "data": [go.Scatter(y=energies, name="energy"),
# #                                 go.Scatter(y=np.ones(len(energies)) * thres, 
# #                                            name="thres")]})
# # play the initial and the generated files in notebook:
# # IPython.display.display(IPython.display.Audio("D:/PhD/audio_recorder_downloads_wavs/Tarcutta_April_May_2021/WetA/20210427_AAO/data/20210427T000000+1000_REC.wav"))
# # IPython.display.display(IPython.display.Audio("D:/PhD/audio_recorder_downloads_wavs/Tarcutta_April_May_2021/WetA/20210427_AAO/data/20210427T000000+1000_REC_processed.wav"))

: 

Split .wav file into 5-second long non-overlapping segments

In [ ]:
from pydub import AudioSegment 
from pydub.utils import make_chunks 

myaudio = AudioSegment.from_file(file, "wav") 
chunk_length_ms = 60000 # pydub calculates in millisec (60 sec)
chunks = make_chunks(myaudio,chunk_length_ms) #Make chunks of one sec 
for i, chunk in enumerate(chunks): 
    chunk_name = "D:/PhD-data/tarcutta-half-day/60 sec chunks/audio_segment{0}.wav".format(i) 
    print ("exporting", chunk_name) 
    chunk.export(chunk_name, format="wav") 

: 

Iterate through folders of audio files and generate a copy of the folder structure with all audio files split into 4.5 second segments

In [ ]:
geo_path = "D:\\esc\\data\\geophony"
anthro_path = "D:\\esc\\data\\anthrophony"
bio_path = "D:\\esc\\data\\biophony"
other_path = "D:\\esc\\data\\other"
output_path = "D:\\esc\\processeddata"

file_paths = [geo_path, anthro_path, bio_path, other_path]

: 

In [ ]:
import os
from pydub import AudioSegment

# define the directory containing the subdirectories
parent_dir = "D:\\PhD-data\\tarcutta-half-day"

# loop through each subdirectory
for subdir in os.listdir(parent_dir):
    # construct the path to the subdirectory
    subdir_path = os.path.join(parent_dir, subdir)
    
    # check if the subdirectory is a directory
    if os.path.isdir(subdir_path):
        # loop through each .wav file in the subdirectory
        for filename in os.listdir(subdir_path):
            # construct the path to the .wav file
            filepath = os.path.join(subdir_path, filename)
            
            # check if the file is a .wav file
            if filename.endswith(".wav"):
                # load the audio file using pydub
                audio = AudioSegment.from_file(filepath, format="wav")
                
                # split the audio file into 4.5 second non-overlapping chunks
                chunk_length_ms = 4500
                chunks = [audio[i:i+chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]
                
                # remove any chunks that are less than 4.5 seconds long
                chunks = [chunk for chunk in chunks if len(chunk) == chunk_length_ms]
                
                # check if there are any chunks left
                if chunks:
                    # loop through each chunk and save it as a new .wav file
                    for i, chunk in enumerate(chunks):
                        chunk_filename = f"{os.path.splitext(filename)[0]}_{i+1}.wav"
                        chunk_filepath = os.path.join(subdir_path, chunk_filename)
                        chunk.export(chunk_filepath, format="wav")
                        
                # remove the file if it's less than 4.5 seconds long or cannot be split evenly into 4.5-second chunks
                else:
                    os.remove(filepath)


: 

In [ ]:
import os
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import numpy as np
import pandas as pd
from pydub import AudioSegment 
from pydub.utils import make_chunks 

path = "D:\\PhD-data\\tarcutta-half-day"
# for path in file_paths:
for subdir, dirs, files in os.walk(path):
    for file in files:
        filepath = subdir + os.sep + file

        if filepath.endswith(".wav"):
            print(filepath)
            myaudio = AudioSegment.from_file(filepath, "wav") 
            chunk_length_ms = 4500 # pydub calculates in millisec (4.5 sec)
            chunks = make_chunks(myaudio,chunk_length_ms) #Make chunks of 4.5 sec 
            for i, chunk in enumerate(chunks):
                chunk_name = "chunk{}{}.wav".format(file, i) 
                print ("exporting", chunk_name)
                chunk.export(subdir + "\\" + chunk_name, format="wav") 

: 

In [ ]:
import os
import wave

def split_audio_files(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all .wav files in the input folder
    audio_files = [f for f in os.listdir(input_folder) if f.endswith('.wav')]

    for file in audio_files:
        file_path = os.path.join(input_folder, file)
        
        # Open the audio file
        with wave.open(file_path, 'rb') as audio:
            frame_rate = audio.getframerate()
            num_channels = audio.getnchannels()
            bytes_per_sample = audio.getsampwidth()

            # Calculate the number of frames for a 4.5-second duration
            frames_per_chunk = int(frame_rate * 4.5)

            # Calculate the number of frames for the whole audio file
            total_frames = audio.getnframes()

            # Calculate the total number of chunks
            total_chunks = total_frames // frames_per_chunk

            total_time = 0
            # Read and split the audio file into chunks
            for chunk_index in range(total_chunks):
                # Read the frames for the current chunk
                frames = audio.readframes(frames_per_chunk)
    
                # Create a new file name for the chunk
                chunk_file_name = f'{os.path.splitext(file)[0]}_{total_time}-{total_time + 4.5}.wav'
                chunk_file_path = os.path.join(output_folder, chunk_file_name)

                total_time += 4.5
                # Write the chunk frames to a new .wav file
                with wave.open(chunk_file_path, 'wb') as chunk_audio:
                    chunk_audio.setnchannels(num_channels)
                    chunk_audio.setsampwidth(bytes_per_sample)
                    chunk_audio.setframerate(frame_rate)
                    chunk_audio.writeframes(frames)

        print(f'File {file} split into {total_chunks} chunks.')

split_audio_files('E:\\audio_recorder_downloads_wavs\\Tarcutta_April_May_2021\\DryA\\20210427_AAO', 'D:\\PhD-data\\Tarcutta-DryA')


: 

In [ ]:
import os
import wave

def split_audio_files(input_folder, output_folder, duration):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all .wav files in the input folder
    audio_files = [f for f in os.listdir(input_folder) if f.endswith('.flac') or f.endswith('.wav')]

    for file in audio_files:
        file_path = os.path.join(input_folder, file)
        
        # Open the audio file
        with wave.open(file_path, 'rb') as audio:
            frame_rate = audio.getframerate()
            num_channels = audio.getnchannels()
            bytes_per_sample = audio.getsampwidth()

            # Calculate the number of frames for a 4.5-second duration
            frames_per_chunk = int(frame_rate * 4.5)

            # Calculate the number of frames for the whole audio file
            total_frames = audio.getnframes()

            # Calculate the total number of chunks
            total_chunks = total_frames // frames_per_chunk

            total_time = 0
            duration_seconds = duration * 60  # Convert duration from minutes to seconds

            # Read and split the audio file into chunks
            for chunk_index in range(total_chunks):
                # Read the frames for the current chunk
                frames = audio.readframes(frames_per_chunk)

                # Break out of the loop if the total_time exceeds the duration_seconds
                if total_time > duration_seconds:
                    break
    
                # Create a new file name for the chunk
                chunk_file_name = f'{os.path.splitext(file)[0]}_{total_time}-{total_time + 4.5}.wav'
                chunk_file_path = os.path.join(output_folder, chunk_file_name)

                total_time += 4.5
                # Write the chunk frames to a new .wav file
                with wave.open(chunk_file_path, 'wb') as chunk_audio:
                    chunk_audio.setnchannels(num_channels)
                    chunk_audio.setsampwidth(bytes_per_sample)
                    chunk_audio.setframerate(frame_rate)
                    chunk_audio.writeframes(frames)

        print(f'File {file} split into {total_chunks} chunks.')

split_audio_files('D:\\PhD-data\\testers', 'D:\\PhD-data\\test', duration=20)

: 

latest below 02-11-23 

In [7]:
import os
from pydub import AudioSegment

def split_audio_files(input_folder, output_folder, duration):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all audio files in the input folder
    audio_files = [f for f in os.listdir(input_folder) if f.endswith('.wav') or f.endswith('.mp3') or f.endswith('.flac')]

    for file in audio_files:
        file_format = file.rsplit('.', 1)[1]
        if file_format == "flac":
            file_format = "flac"  # correct format name for flac
        elif file_format == "mp3":
            file_format = "mp3"
        elif file_format == "wav":
            file_format = "wav"
        else:
            raise ValueError(f"Unsupported file format: {file_format}")
        file_path = os.path.join(input_folder, file)
        # Load the audio file with pydub
        audio = AudioSegment.from_file(file_path)

        frame_rate = audio.frame_rate
        duration_seconds = duration * 60  # Convert duration from minutes to seconds
        chunk_length_ms = 4500  # Length of each audio chunk in milliseconds

        total_length_ms = len(audio)
        total_chunks = total_length_ms // chunk_length_ms
        total_time_ms = 0

        for chunk_index in range(total_chunks):
            if total_time_ms > duration_seconds * 1000:
                break

            start_time = chunk_index * chunk_length_ms
            end_time = start_time + chunk_length_ms
            chunk_data = audio[start_time:end_time]

            # Use the file_format variable instead of slicing the original filename
            chunk_file_name = f'{os.path.splitext(file)[0]}_{total_time_ms//1000}-{(total_time_ms + chunk_length_ms)//1000}.{file_format}'
            chunk_file_path = os.path.join(output_folder, chunk_file_name)

            # Export using the determined file format
            chunk_data.export(chunk_file_path, format=file_format)
            total_time_ms += chunk_length_ms

        print(f'File {file} split into {total_chunks} chunks.')

# Example call
split_audio_files('D:\\PhD-data\\testers', 'D:\\PhD-data\\test', duration=20)

File 20210203T033909+1000_Daintree-Rainforest-Observatory-Dry-B_489445.flac split into 276 chunks.
